In [1]:
# -*- coding: utf-8 -*-
# --------------------------------------------------
#
# conditionalRandomFields2.ipynb
#
# using BERT's token and sentence features
#
# Written by cetinsamet -*- cetin.samet@metu.edu.tr
# April, 2019
# --------------------------------------------------

In [2]:
from seqeval.metrics import classification_report
from sklearn_crfsuite import CRF
import scipy.io as sio
from tqdm import tqdm
import numpy as np
import pickle

In [3]:
def readFile(filepath):
    
        text     = []
        sentence = []

        with open(filepath, 'r') as infile:
            for line in infile:
                word, _, _, _ = line.strip().split('\t')

                if word == '<S>':
                    text.append(sentence)
                    sentence = []
                    continue

                sentence.append(line.strip())

        return text

In [4]:
trainText = readFile('data/train.txt')
validText = readFile('data/valid.txt')
testText  = readFile('data/test.txt') 

In [5]:
features  = sio.loadmat('data/features.mat')

In [ ]:
trainFeatures = []
trainLabels   = []

feats         = []
labels        = []

tokenIdx      = 0
sentenceIdx   = 0

for sentence in tqdm(trainText):
    
    for token in sentence:
        _, _, _, label = token.split('\t')
        feat = np.concatenate((features['trainTokenFeatures'][tokenIdx], features['trainSentFeatures'][sentenceIdx]))
        d = dict(enumerate(feat))
        d = {str(k): v for k, v in d.items()}
        feats.append(d)
        tokenIdx += 1
        labels.append(label)
        
    trainFeatures.append(feats)
    trainLabels.append(labels)
    feats, labels = [], []
    sentenceIdx += 1

In [ ]:
validFeatures = []
validLabels   = []

feats         = []
labels        = []

tokenIdx      = 0
sentenceIdx   = 0

for sentence in tqdm(validText):
    
    for token in sentence:
        _, _, _, label = token.split('\t')
        feat = np.concatenate((features['validTokenFeatures'][tokenIdx], features['validSentFeatures'][sentenceIdx]))
        d = dict(enumerate(feat))
        d = {str(k): v for k, v in d.items()}
        feats.append(d)
        tokenIdx += 1
        labels.append(label)
        
    validFeatures.append(feats)
    validLabels.append(labels)
    feats, labels = [], []
    sentenceIdx += 1

In [ ]:
testFeatures = []
testLabels   = []

feats         = []
labels        = []

tokenIdx      = 0
sentenceIdx   = 0

for sentence in tqdm(testText):
    
    for token in sentence:
        _, _, _, label = token.split('\t')
        feat = np.concatenate((features['testTokenFeatures'][tokenIdx], features['testSentFeatures'][sentenceIdx]))
        d = dict(enumerate(feat))
        d = {str(k): v for k, v in d.items()}
        feats.append(d)
        tokenIdx += 1
        labels.append(label)
        
    testFeatures.append(feats)
    testLabels.append(labels)
    feats, labels = [], []
    sentenceIdx += 1

In [ ]:
trainvalFeatures = trainFeatures + validFeatures
trainvalLabels   = trainLabels   + validLabels

In [7]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True)

In [ ]:
crf.fit(trainvalFeatures, trainvalLabels)

In [ ]:
# SAVE CONDITIONAL RANDOM FIELDS MODEL
with open('model/conditional_random_fields2.pickle', 'wb') as outfile:
    pickle.dump(crf, outfile, pickle.HIGHEST_PROTOCOL)
    print("Conditional Random Fields model is saved.")

In [7]:
# LOAD CONDITIONAL RANDOM FIELDS MODEL
with open('model/conditional_random_fields2.pickle', 'rb') as infile:
    crf = pickle.load(infile)

In [ ]:
trainPredLabels = crf.predict(trainFeatures)

print("### TRAIN CLASSIFICATION REPORT ###\n")
print(classification_report(trainLabels, trainPredLabels))

In [ ]:
validPredLabels = crf.predict(validFeatures)

print("### VAL CLASSIFICATION REPORT ###\n")
print(classification_report(validLabels, validPredLabels))

In [8]:
testPredLabels  = crf.predict(testFeatures)

print("### TEST CLASSIFICATION REPORT ###\n")
print(classification_report(testLabels, testPredLabels))

### TEST CLASSIFICATION REPORT ###

              precision    recall  f1-score   support

ORGANIZATION       0.58      0.36      0.44       862
      PERSON       0.61      0.38      0.47      1594
       MONEY       0.77      0.54      0.64       113
     PERCENT       0.99      0.94      0.97       107
    LOCATION       0.70      0.50      0.58      1091
        DATE       0.80      0.62      0.70       364
        TIME       0.91      0.87      0.89        23

   micro avg       0.67      0.45      0.54      4154
   macro avg       0.66      0.45      0.53      4154

